In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import os
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
# constants and hyperparameters
MAX_WORD_INDEX = 10000
EMBEDDING_DIM = 128
BATCH_SIZE = 128
NUM_EPOCHS = 10
NUM_LSTM_UNITS = 32
DROPOUT_RATE = 0.2
LR = 0.001
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1.0e-8
DECAY = 0.0
VAL_PERC = 0.4

In [3]:
if(os.path.isdir(os.getcwd()+'/dataset/processed') == False):
    species = pd.read_csv('class_names.csv').values.tolist()
    species.insert(0,['Alfafa'])
    i = 0
    os.makedirs(os.getcwd()+'/dataset/processed', exist_ok=True)
    for dirname, _, filenames in os.walk(os.getcwd()+'/dataset/resized'):
        cur_specie = species[i][0]
        class_dir = os.getcwd()+'/dataset/processed/'+cur_specie
        os.makedirs(class_dir, exist_ok=True)
        img_count = 1
        for filename in filenames:
            image = Image.open(os.path.join(dirname, filename))
            img_width, img_height = image.size
            if(img_width == img_height):
                image = image.resize((256,256))
            elif(img_width > img_height):
                image = image.resize((img_width, 256)).crop((0,0,256,256))
            else:
                image = image.resize((256, img_height)).crop((0,0,256,256))
            image.save(class_dir+'/'+cur_specie+str(img_count)+'.png')
            img_count += 1
        if(dirname != os.getcwd()+'/dataset/resized'):
            i += 1

In [4]:
def loadDataset(path):
    data = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            image = Image.open(os.path.join(dirname, filename))
            data.append([pd.array(image.getdata()),dirname])
    return pd.DataFrame(data)
            
    

In [16]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
dataset = image_dataset_from_directory(os.getcwd()+'/dataset/processed', 
                                       labels='inferred',
                                       label_mode='categorical',
                                       batch_size=256)
print(dataset)

Found 16526 files belonging to 35 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 35), dtype=tf.float32, name=None))>


In [ ]:
dataset = dataset.unbatch()
images = list(dataset.map(lambda x, y:x))
labels = list(dataset.map(lambda x, x:y))

In [17]:
from random import seed
from random import randint
from datetime import datetime
# seed random number generator
seed(datetime.now())
rand = randint(0,16526)
Image.fromarray(dataset[0][rand]).show()
print(dataset[1][rand])

ModuleNotFoundError: No module named 'tensorflow'

In [8]:
model_name = 'model_cnn'

history_file = os.path.join(model_name, f'history_{model_name}.csv')
logdir = os.path.join(model_name, 'log')
ckpts = os.path.join(model_name, 'ckpts')
os.makedirs(logdir, exist_ok=True)
os.makedirs(ckpts, exist_ok=True)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dataset[0],dataset[1],test_size = 0.4)

del dataset

#reashaping and normalizing
#X_train = np.array(X_train/255.0)
#X_test = np.array(X_test/255.0)



In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

# build model
model = Sequential()

model.add(Conv2D(64,(3,3),input_shape = (256,256,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(70))
model.add(Dense(35, activation='sigmoid'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 127, 127, 64)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 125, 125, 32)      18464     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 123008)            0         
                                                                 
 dense_4 (Dense)             (None, 70)               

In [32]:
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

# set optimizer
opt = optimizers.Adam(learning_rate=LR,
    beta_1=BETA1,
    beta_2=BETA2,
    epsilon=EPSILON,
    decay=DECAY)

In [33]:
# set loss and metrics
loss = losses.binary_crossentropy
met = [metrics.binary_accuracy]

In [34]:
# compile model: optimization method, training criterion and metrics
model.compile(
  optimizer=opt,
  loss=loss,
  metrics=met
)

In [35]:
# early stop, save best checkpoint
filepath = ckpts + '/weights-improvement-{epoch:02d}-{val_binary_accuracy:.4f}.hdf5'
callbacks_list = [
  EarlyStopping(
    monitor='binary_accuracy',
    patience=10),
  ModelCheckpoint(
    filepath=filepath,
    monitor='val_binary_accuracy',
    save_best_only=True,
    verbose=1),
  TensorBoard(
    log_dir=logdir),
    ]

In [36]:
# split training data into training and validation
nsamples = X_train.shape[0]
nval_samples = int(VAL_PERC * nsamples)
X_val = X_train[:nval_samples]
partial_X_train = X_train[nval_samples:]
y_val = y_train[:nval_samples]
partial_y_train = y_train[nval_samples:]

In [68]:
##print("X", partial_X_train)
##print("y", partial_y_train)

In [69]:
# train model
history = model.fit(partial_X_train,
  partial_y_train,
  epochs=NUM_EPOCHS,
  batch_size=BATCH_SIZE,
  validation_data=(X_val, y_val),
  callbacks=callbacks_list,
  verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).